# Channel Break Out Detection

## Import Data
##### 1. Determinar puntos de cambios de tendencia
##### 2. Con los valores high determinar la linea de tendencia bajista
##### 3. Determinar el punto de ruptura del canal, mas que todo la tendcia techo (canal bajista)
##### 4. Determinar manipulacion del mercado y falsa ruptura

In [50]:
import pandas as pd
import numpy as np
#import plotly.graph_objects as go
import yfinance as yf
#import matplotlib.pyplot as plt
#import math
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
#from bokeh.plotting import figure, show, column
from sklearn.linear_model import LinearRegression

In [2]:
#df = pd.read_csv(r"D:\CARLOS\TRADING\EURUSD_Candlestick_1_D_BID_04.05.2003-21.01.2023.csv")
ini2 = date.today() - timedelta(days=728) # 2 years ago
today = date.today()
today_str = str(today)
ini2_str = str(ini2)
tickers = [
    'SPY',
    'META',
    'AAPL'
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [5]:
df = pd.DataFrame()
for ticker in tickers:
    company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    company['datetime'] = pd.to_datetime (company.index)
    company['companyName'] = ticker
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)
    
    df = pd.concat([df, company],ignore_index=True)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## Detect Pivots/Fractals

In [8]:
ord=10
#for ticker in tickers:
#print("====", ticker)
#dfpl = df[(df.companyName==ticker)] #df.query("companyName=='AAPL'")
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]
#print(max_idx)
#print(min_idx)
#df['pivotHigh'] = np.nan
#df['pivotLow'] = np.nan
#df['isPivot'] = np.nan
#df['isPivot'] = np.nan
# Aplicar el cálculo solo a los índices en la lista
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

###### Canales bajistas, cuando el PM de 40 esta por encima del PM de 20 en direccion desendente
###### RSI>70 y sobre las olas de Bollinger

In [10]:
def rcb01(df):
    if ((df['pivotLow']>0) & (df['SMA40']>df['SMA20'])):
    #if ((df['SMA40']>df['SMA20']) & (df['RSI']<=35) & (df['lower_band']>df['Low'])):
        return 1
    else:
        return 0
df['rcb01_01'] = df.apply(rcb01, axis=1)

## Detect Price Channel

In [13]:
def collect_channel(candle, backevalTrend, trendH):
    #localdf = df[candle-backcandles-window:candle-window]
    localdf = dfpl.loc[candle-backevalTrend:candle] #tomar en cuenta el backcandles
    #localdf['isPivot'] = localdf.apply(lambda x: isPivot(x.name,window), axis=1)
    
    #highs = localdf.High.values
    #idxhighs = localdf.High.index
    #lows = localdf.Low.values
    #idxlows = localdf.Low.index

    highs = localdf.loc[localdf.index.isin(trendH)].High.values
    idxhighs = localdf.loc[localdf.index.isin(trendH)].High.index
    #lows = localdf.loc[localdf.index.isin(trendL)].Low.values
    #idxlows = localdf.loc[localdf.index.isin(trendL)].Low.index

    #highs = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 1)].High.values
    #idxhighs = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 1)].High.index
    #lows = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 2)].Low.values
    #idxlows = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 2)].Low.index

    #lows = localdf.loc[localdf.index.isin(trendL)].Low.values
    #idxlows = localdf.loc[localdf.index.isin(trendL)].Low.index

    #print("highs:",highs)
    #print("idxhighs:",idxhighs)
    #print("lows:",lows)
    #print("idxlows:",idxlows)
    #print ("tamanio:",localdf.shape[0])
    
    #if len(lows)>=2 and len(highs)>=2:
    if len(highs)>=2:
        #sl_lows, interc_lows, r_value_l, _, _ = stats.linregress(idxlows,lows)
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)    
        #return(sl_lows, interc_lows, sl_highs, interc_highs, r_value_l**2, r_value_h**2)
        return(sl_highs, interc_highs, r_value_h**2)
    else:
        return(0,0,0)
    

In [15]:
def backcandle(candle):
    
    trendH = []
    backeval = 0
    val = 0
    
    dfpl['PM40_PM20'] = dfpl['SMA40']-dfpl['SMA20']

    if (dfpl.loc[candle,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:candle, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4
        val = candle - backeval
    
    #print("idx_negativo:",idx_negativo)
    
    if val>75:
        backeval=candle-75
        
    if (dfpl.loc[backeval,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:backeval, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4

    if val>75:
        backeval=candle-75

    #ini = candle - backeval
    ini = backeval
    trendprev=dfpl.loc[ini:candle]
    trend = trendprev.reset_index(drop=False)

    idxhighs = trend.index.to_numpy().reshape(-1,1) # Convertir X a una matriz 2D
    highs = trend.High.values
    modelo1 = LinearRegression()
    modelo1.fit(idxhighs, highs)
    # Obtener la predicción de la línea de regresión
    Y_pred = modelo1.predict(idxhighs)
    # Calcular la distancia solo de los puntos que están POR ENCIMA de la línea
    trend['Distancia'] = trend["High"] - Y_pred  # Diferencia entre el valor real y la predicción
    # Filtrar los puntos que están por encima (donde Y > Y_pred, es decir, Distancia > 0)
    df_encima = trend[trend['Distancia'] > 0]
    puntos_mas_lejanos = df_encima.nlargest(2, 'Distancia')   #2 puntos mas lejanos
    trendH = puntos_mas_lejanos["index"].tolist()
    trendH.append(candle)   
    
    return trendH, backeval

## Detect Break Out

In [18]:
def trendChannel(row,candle, backeval, window, sl_highs, interc_highs):

    if (candle-backeval-window)<0:
        return np.nan

    #print("=== DATOS ===")
    #print("candle:",candle)
    #print("backcandles:",backcandles)
    #print("window:",window)
    #trendprevlow = (sl_lows*row['id'].shift(1).fillna(0) + interc_lows)
    #trendprevhigh = (sl_highs*row['id'].shift(1).fillna(0) + interc_highs)

    #x = np.array(range(candle-backcandles-window, candle+window))
    #trendcurrlow = np.where(np.logical_or((row.index > (candle+window)),  (row.index <  candle-backcandles-(window+3))), np.nan, (sl_lows*row.index + interc_lows))
    #trendcurrhigh = (sl_highs*row['id'] + interc_highs)
    ini=candle-backeval
    fin=candle+window
    #print("candle:",candle,",backeval", backeval, ", ini:", ini, ", fin:", fin,   ", rowMin:", row.index[0],", rowMax:", row.index[-1])
    trendcurrhigh = np.where(np.logical_or((row.index > fin),  (row.index <  ini)), np.nan, (sl_highs*row.index + interc_highs))
    return trendcurrhigh

In [20]:
def isBreakOut(candleEval,backcandles, window, start, candle):
    if (candleEval-backcandles-window)<0:
        return 0

    if candleEval==start:
        prev_idx = candleEval
    else:
        prev_idx = candleEval-1
        
    prev_high = dfpl.loc[prev_idx].High
    prev_low = dfpl.loc[prev_idx].Low
    prev_close = df.loc[prev_idx].Close
    
    curr_idx = candleEval
    curr_high = dfpl.loc[candleEval].High
    curr_low = dfpl.loc[candleEval].Low
    curr_close = dfpl.loc[candleEval].Close
    curr_open = dfpl.loc[candleEval].Open

    #trend_prevlow=dfpl.loc[prev_idx].trendcurrlow
    trend_prevhigh=dfpl.loc[prev_idx].trendcurrhigh

    #trend_currlow=dfpl.loc[candleEval].trendcurrlow
    trend_currhigh=dfpl.loc[candleEval].trendcurrhigh

    #codigo para tendencia LOW, nos importa breaks en tendencia HIGH
    #if ( prev_high > trend_prevlow and
    #    prev_close < trend_prevlow and
    #    curr_open < trend_currlow and
    #    curr_close < trend_currlow and candleEval>candle): #and r_sq_l > 0.9
    #    return 1
    
    if ( prev_low < trend_prevhigh and
        prev_close > trend_prevhigh and
        curr_open > trend_currhigh and
        curr_close > trend_currhigh and
        candleEval>candle): #and r_sq_h > 0.9
        return 2

    elif (curr_open > trend_currhigh and
        curr_close > trend_currhigh and candleEval>candle):
        return 3
    
    else:
        return np.nan

In [22]:
def breakpointpos(x):
    if x['isBreakOutIni'] in (2,3):
        return x['Low']+3e-3
    elif x['isBreakOutIni']==1:
        return x['High']-3e-3
    else:
        return np.nan

##### Hallando casos de Ruptura del Canal Bajista

In [29]:
def revisionVelas(dfpl, backeval,window, candle):
    return 1 

In [31]:
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []
cant = 0
for i, row in df.iterrows():
    #if (i<=200):
    if(df['rcb01_01'][i]==1): #posibles casos 
        #print("posible caso:",i, df['companyName'][i])
        cant = 0
        candle = i
        cnt = 0
        valiniHigh = df.loc[candle,"High"]
        #valiniLow = df.loc[candle,"Low"]
        ticker = df['companyName'][i]
        #Reinicio de casos por company
        if ticker2 != ticker:
            ticker2 = ticker
            caso = 0
        dfpl = df[(df.companyName==ticker)].loc[i-backeval:i+backeval]       

        #print("rowMin:", dfpl.index[0],", rowMax:", dfpl.index[-1])
                    
        h=1
        l=1

        cant = cant +1
        cantHorasTrend=0
        #print("opcionH:",h,"opcionL:",l)
        trendH, backevalTrend = backcandle(candle)
        #print ("-->trendH:",trendH)
        #print ("-->backevalTrend:",backevalTrend)    
        
        cantHorasTrend = (dfpl.loc[candle,"datetime"] - dfpl.loc[backevalTrend,"datetime"]).total_seconds() / 3600

        if (cantHorasTrend>48): #Verificacion de caida por lo menos 2 dias
            backevalTrend2 = candle - backevalTrend
            #print ("backevalTrend2", backevalTrend2)
                        
            #revision de trend negativo indica que esta cayendo, solo me quedo con estos casos
            sl_highs, interc_highs, r_sq_h = collect_channel(candle, backevalTrend2, trendH)
                    
    
            #print("======:", candle)
            #print("===trendH===")
            #print(trendH)
            #print("===trendL===")
            #print(trendL)
            dfpl.loc[trendH, "trendH"] = 1
        
            
            #print("sl_highs:", sl_highs, ", sl_lows:", sl_lows, ", r_sq_h:", r_sq_h)
            if (sl_highs<=0): #solo tendencias a la baja and ind_vela>0 sl_lows<0 and r_sq_h>=0.7  and sl_highs<=-0.1 and ind_vela>0
                
                #crear linea de tendencia high y low
                dfpl['trendcurrhigh']  = trendChannel(dfpl,candle,backevalTrend2,window,sl_highs,interc_highs)
                
                ind_vela = revisionVelas(dfpl, backeval,window, candle)
                promVol = dfpl.loc[i-backeval:i]["Volume"].mean()
    
                #print("ind_vela:",ind_vela)
                if (ind_vela>0):
                    dfpl["bearishSlope"] = np.nan #pendiente bajista en vacio
                    dfpl.loc[i, "bearishSlope"] = 1 #pendiente bajista solo al punto evaluado            
                    dfpl["ind_posicion"] = 0
                    dfpl.loc[(dfpl.index < i), 'ind_posicion'] = -1
                    dfpl.loc[(dfpl.index > i), 'ind_posicion'] = 1
                    dfpl["promVol"] = promVol
                    dfpl["sl_highs"] = sl_highs
                    dfpl["r_sq_h"] = r_sq_h
            
                    start = dfpl.index[0]
                    dfpl["isBreakOut"] = [isBreakOut(candleEval, backevalTrend2, window, start, candle) for candleEval in dfpl.index]
                    # Solo me quedo con el primer BREAK OUT
                    cnt = dfpl.query("isBreakOut in (2,3) and Close>Open").shape[0] 
                    id=0
                    id2=0
                    if cnt>0:
                        id = dfpl.query("isBreakOut in (2,3) and Close>Open").index[0]                        
                        #dfpl['isBreakOutIni'] = np.where(id!=dfpl.index, np.nan, dfpl['isBreakOut'])                        
                        dfpl.loc[id,'isBreakOutIni'] = 1
                        dfpl['breakpointpos'] = dfpl.apply(lambda row: breakpointpos(row), axis=1)
                        cnt2 = dfpl.query("index>@id and isPivot==1").shape[0]
                        
                        if cnt2>0:
                            id2 = dfpl.query("index>@id and isPivot==1").index[0]
                            dfpl.loc[id2,'isBreakOutFinal'] = 1
                        else:
                            dfpl['isBreakOutFinal'] = np.nan                      
                        
                    else:
                        dfpl['isBreakOutIni'] = np.nan                        
                        dfpl['breakpointpos'] = np.nan
            
                    if cnt>0:
                        #INSERT CASO
                        caso=caso+1                            
                        dfpl["caso"] = caso
                        print(i, df['companyName'][i])
                        if len(df_casos)<1:
                            df_casos = dfpl
                        else:
                            df_casos = pd.concat([df_casos, dfpl],ignore_index=False)


68 SPY
167 SPY
356 SPY
411 SPY
761 SPY
804 SPY
959 SPY
1285 SPY
1713 SPY
1989 SPY
2248 SPY
2266 SPY
2311 SPY
2454 SPY
2473 SPY
2588 SPY
2745 SPY
2933 SPY
2965 SPY
3014 SPY
3053 SPY
3292 SPY
3424 SPY
3460 SPY
3599 META
3616 META
3964 META
4028 META
4064 META
4253 META
4285 META
4401 META
4431 META
4606 META
5098 META
5165 META
5252 META
5293 META
5436 META
5568 META
5685 META
5746 META
5918 META
5954 META
6150 META
6169 META
6225 META
6236 META
6298 META
6452 META
6490 META
6545 META
6737 META
6772 META
6803 META
6851 META
6904 META
6940 META
7023 AAPL
7129 AAPL
7380 AAPL
7510 AAPL
7524 AAPL
7538 AAPL
7697 AAPL
7708 AAPL
7768 AAPL
7887 AAPL
7911 AAPL
8075 AAPL
8087 AAPL
8198 AAPL
8373 AAPL
8534 AAPL
8618 AAPL
8631 AAPL
8655 AAPL
8683 AAPL
8701 AAPL
8746 AAPL
9202 AAPL
9225 AAPL
9258 AAPL
9565 AAPL
9701 AAPL
9978 AAPL
10013 AAPL
10049 AAPL
10152 AAPL
10259 AAPL
10432 AAPL


In [43]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
import os
path=r'D:\PROYECTO_CARLOS\Notebooks\notebooks\data\cb_h.txt'
#path = r'C:\Users\carlo\OneDrive\Documentos\TRADER\traderapp\data\rcb_h.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

File does not exists. File needs to be created.


##### BACKTESTING
###### Usando paquete backtesting.py

In [52]:
from backtesting import Backtest, Strategy

C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [54]:
# =============== BACKTESTING ================= #
# obteniendo dataframe backtesting
df_backTesting = pd.merge(df, df_casos[['datetime','companyName', 'isBreakOutIni','isBreakOutFinal','caso']], on = ['companyName','datetime'], how='left')
# datos de trade se asigna una vela anterior debido a que backtesting.py toma entra como la siguiente vela
df_backTesting['isBreakOutIni'] = df_backTesting['isBreakOutIni'].shift(-1)
df_backTesting['isBreakOutFinal'] = df_backTesting['isBreakOutFinal'].shift(-1)
#modificamos indice con el compo datetime
df_backTesting.set_index('datetime', inplace=True)

In [56]:
#Strategia
class strategyRupturaCanal(Strategy):
    def init(self):
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) 

    def next(self):
        if self.breakout_entry[-1] == 1:
            self.buy()  #COMPRA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE

In [60]:
# Por cada TIcket
my_stats = {}
for ticker in df_backTesting['companyName'].unique():
    ticker_data = df_backTesting[df_backTesting['companyName'] == ticker] #Filtro por cada ticker
    #print("\n" + "="*50 + "\n")
    bt = Backtest(ticker_data, strategyRupturaCanal, cash=10_000)
    #stats = bt.run()
    my_stats[f"stats_BreakOUT_{ticker}"] = bt.run()
    # Mostramos
    print(f"Resultados para {ticker}:")
    print(stats)   
    #plt.title(f"Backtest -  {ticker}")
    bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/4511 [00:00<?, ?bar/s]

C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=3023: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=3060: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Resultados para SPY:
<module 'scipy.stats' from 'C:\\Users\\Linder\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


Backtest.run:   0%|          | 0/5591 [00:00<?, ?bar/s]

C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=1223: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=4203: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Resultados para META:
<module 'scipy.stats' from 'C:\\Users\\Linder\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


Backtest.run:   0%|          | 0/5580 [00:00<?, ?bar/s]

C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=776: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=1914: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=2807: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=3192: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Linder\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=3992: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Resultados para AAPL:
<module 'scipy.stats' from 'C:\\Users\\Linder\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\Linder\AppData\Local\Temp\ipykernel_2224\2633193108.py:13: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


In [62]:
#my_stats['stats_BreakOUT_SPY']

Start                     2023-04-13 13:30...
End                       2025-04-09 19:30...
Duration                    727 days 06:00:00
Exposure Time [%]                    18.24706
Equity Final [$]                  17099.06839
Equity Peak [$]                   17173.15884
Return [%]                           70.99068
Buy & Hold Return [%]                30.23837
Return (Ann.) [%]                    31.04429
Volatility (Ann.) [%]                10.76004
CAGR [%]                               20.428
Sharpe Ratio                          2.88515
Sortino Ratio                        13.82802
Calmar Ratio                          6.23104
Alpha [%]                            63.58112
Beta                                  0.24504
Max. Drawdown [%]                     -4.9822
Avg. Drawdown [%]                    -0.38305
Max. Drawdown Duration       62 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                   21
Win Rate [%]                      

In [64]:
my_stats['stats_BreakOUT_SPY']['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,Entry_λ(isBreakOu…),Exit_λ(isBreakOu…)
0,24,222,234,412.119995,420.095001,None,None,191.400146,0.019351,2023-05-17 15:30:00+00:00,2023-05-19 13:30:00+00:00,1 days 22:00:00,None,NaN,NaN
1,23,430,480,432.619995,443.529999,None,None,250.930084,0.025218,2023-06-27 13:30:00+00:00,2023-06-30 17:30:00+00:00,3 days 04:00:00,None,NaN,NaN
2,23,541,606,438.390015,455.010010,None,None,382.259888,0.037911,2023-07-10 16:30:00+00:00,2023-07-19 13:30:00+00:00,8 days 21:00:00,None,NaN,NaN
3,24,943,955,440.559998,443.274994,None,None,65.159912,0.006163,2023-09-19 16:30:00+00:00,2023-09-20 15:30:00+00:00,0 days 23:00:00,None,NaN,NaN
4,25,1032,1050,424.385010,431.200012,None,None,170.375061,0.016059,2023-09-27 18:30:00+00:00,2023-09-29 13:30:00+00:00,1 days 19:00:00,None,NaN,NaN
5,26,1217,1264,410.404999,437.035004,None,None,692.380127,0.064887,2023-10-27 19:30:00+00:00,2023-11-07 18:30:00+00:00,10 days 23:00:00,None,NaN,NaN
6,25,1544,1561,469.959991,475.309998,None,None,133.750153,0.011384,2024-01-08 15:30:00+00:00,2024-01-10 18:30:00+00:00,2 days 03:00:00,None,NaN,NaN
7,23,1970,1990,514.460022,518.510010,None,None,93.149719,0.007872,2024-04-05 13:30:00+00:00,2024-04-09 19:30:00+00:00,4 days 06:00:00,None,NaN,NaN
8,23,2248,2250,520.780029,529.020020,None,None,189.519775,0.015822,2024-05-31 18:30:00+00:00,2024-06-03 13:30:00+00:00,2 days 19:00:00,None,NaN,NaN
9,22,2582,2585,543.210022,545.570007,None,None,51.919678,0.004345,2024-07-26 15:30:00+00:00,2024-07-26 16:30:00+00:00,0 days 01:00:00,None,NaN,NaN


In [ ]:
#path = r'D:\CARLOS\TRADING\BACKTESTING\backtesting\data\data_backtesting.txt'
## check whether the file exists
#if os.path.exists(path):
#    # delete the file
#    os.remove(path)
#else:
#    # if the file does not exist.
#    print("File does not exists. File needs to be created.")
#
##export DataFrame to text file
#with open(path, 'a') as f:
#    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
#    df_backTesting.to_csv(path, header=True, index=None, sep='\t', mode='w')
#    #f.write(df_string)